# The 2018 California Assessment of Student Performance and Progress (CAASPP) System
# Result of Smarter Balance Assessments Tests—English language arts/literacy (ELA) and mathematics

In [ ]:
%matplotlib inline

In [ ]:
#import dependencies

import pandas as pd
import numpy as np
import matplotlib.pylab as plt


#use from datetime import datetime, this feature will help in converting String DateTime to Python DateTime (google)
from datetime import datetime

#data visualization library in python on top of matplotlib
import seaborn as sns

#import categoricalDtype for astype i.e (categories, ordered, etc...)
from pandas.api.types import CategoricalDtype

pd.set_option('display.max_rows', 1000) # display all columns and rows
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None) # no truncate
pd.set_option('precision', 2)  # show only two decimal digits
pd.options.display.float_format = '{:20,.2f}'.format  # no scientitic display

import warnings
warnings.filterwarnings('ignore')

# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 15, 8


## Data Source 1: Public Schools

In [ ]:
# Read the data
pubsch = pd.read_csv('pubschls.csv')

# Keep only the relevant columns
keep_cols = ['CDSCode','StatusType', 'County','OpenDate', 'ClosedDate', 'Charter', 'CharterNum', 'FundingType', 'DOC', 'DOCType', 'SOC', 'SOCType', 'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved', 'Virtual', 'Magnet', 'YearRoundYN']
df = pubsch[keep_cols]
pubsch

In [ ]:
pd.set_option("display.max_rows", 1000, "display.max_columns", 1000)

#Distinguishing data between ACTIVE and INACTIVE Districts and cleaning up database to keep only ACTIVE districts
# Keep only the 'Active' districts
#make a new database
df_active = df[df.StatusType == 'Active']

# Cleanup data, remove the missing values row
df_active = df_active.dropna()

# fix rows/columns that display no data and replace 'No Data' with N/A, just like the previous cells...
df_active = df_active.replace('No Data', np.nan)


#make a base to keep only relevant columns
df_active = df_active[['CDSCode', 'StatusType', 'County', 'OpenDate', 'Charter', 'FundingType', 'DOC', 'DOCType', 'SOC', 'SOCType', 'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName', 'Virtual', 'Magnet', 'YearRoundYN']]

#for calculations in dataset come up with a code that will replace YES/NO column with an Integer for easier calculations
# Replace 'Y' with 1 and 'N' with 0 in Yes/No columns
for i in ['Charter', 'Magnet', 'YearRoundYN']:
    df_active[i] = df_active[i].replace({'Y':1, 'N':0})

# Convert column 'OpenDate' to datetime
#use from datetime import datetime, this feature will help in converting String DateTime to Python DateTime (google)
df_active.OpenDate = df_active.OpenDate.astype(np.datetime64)

# Calcuate the duration of operation of a school since the OpenDate to 'now'
df_active['duration'] = (pd.to_datetime('now') - df_active.OpenDate)/np.timedelta64(1, "Y")

# Create cut points to put 'duration' into different groups so they're easier to distinguish
cut_points = [0,10,20,30,40,50,150]

# Create label names for duration of schools
label_names = ['Less than 10 years', '10-20 years', '20-30 years','30-40 years', '40-50 years', 'More than 50 years']

# Categorize the duration length into 6 groups
df_active['duration_group'] = pd.cut(df_active.duration,cut_points,labels=label_names)
cat_type = CategoricalDtype(categories=label_names,ordered=True)

# Convert column 'duration_group' to ordinal data type with order and label names
df_active['duration_group'] = df_active['duration_group'].astype(cat_type)

# Convert other neccessary columns to categorical data type
df_active.FundingType = df_active.FundingType.astype('category')
cat_type = CategoricalDtype(categories=['Elementary','Elementary-High Combination','Intermediate/Middle/Junior High','High School'],ordered=True)
df_active['EILName'] = df_active['EILName'].astype(cat_type)
df_active

In [ ]:
#Now make graphs based on the information provided from the data above
# GRAPH 1: This graph is based on Funding Type was it Directly Funded or locally Funded

ax = sns.countplot(y='FundingType', data=df_active, dodge=False)

ax.set_title('Distribution of Funding Types for Schools in California 2018')

ax.set_ylabel('Institutions')

plt.savefig("Funding.png")

plt.show()


In [ ]:
#This graph based on data above, show what level of Institution was involved the most. 
# GRAPH 2: Educational Instruction Level
ax = sns.countplot(y='EILName', data=df_active)

ax.set_title('Educational Instruction Level for Schools in California 2018')

ax.set_ylabel('Institutions')

plt.savefig("Educationlevel.png")

plt.show()


In [ ]:
#This graph show which institution has been in operation the longest.
# GRAPH 3: Operation Duration

ax = sns.countplot(y='duration_group', data=df_active)

ax.set_title('Distribution of Operation Duration for Schools in California 2018')

ax.set_ylabel('Institutions')

plt.savefig("Duration.png")

plt.show()

## Data source 2: Smarter Balanced Assessment Test

In [ ]:
# Read data

students_all = pd.read_csv('sb_ca2018_all_csv_v3.txt')
students_all

In [ ]:
# Data Cleaning

# Keep the relevant columns
newdf = students_all[['Subgroup ID','Test Id', 'Grade', 'Total Tested with Scores', 'CAASPP Reported Enrollment', 'Students Tested','Students with Scores', 'Mean Scale Score', 'Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Met and Above', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']]
newdf

In [ ]:
# Replace symbol '*' with null
newdf = newdf.replace('*', np.nan)
newdf

# Convert some columns to numeric
num_cols = ['CAASPP Reported Enrollment', 'Students Tested', 'Students with Scores', 'Mean Scale Score', 'Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Met and Above', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']
for i in num_cols:
    newdf[i] = newdf[i].astype(np.float64)

### RESULTS

### All Students

In [ ]:
#Distinguish Testing grades based on 2018 school datasets and put them into a subgroup
# ALL STUDENTS (without any category) (Subgroup ID = 1) in California
df1 = newdf[newdf['Subgroup ID'] == 1]

# Look at the test 'SB - English Language Arts/Math' (Test Id = 1)
df1_reading = df1[df1['Test Id'] == 1]

# Look at the test 'SB - Mathematics' (Test Id = 2)
df1_math = df1[df1['Test Id'] == 2]

# For reading test, per each grade level, calculate the mean of Percentage Standard Exceeded, Met, Nearly Met or Not Met

# Exclude Grade ID 13 because it presents 'All grades'. We will calculate the average of all grades later

reading_mean = df1_reading[df1_reading.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()

# Add a row 'All' as the average of performance of across all grades
reading_mean.loc['All'] = reading_mean.mean()
reading_mean

In [ ]:
# Do the same for Math  test again excluding Grade ID 13 because it shows 'All Grades'
math_mean = df1_math[df1_math.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()
math_mean.loc['All'] = math_mean.mean()
math_mean

In [ ]:
# need to write this line 2 times because it doesn't take the first command (a bug from plt library) 
plt.rcParams['figure.figsize'] = 20, 10

In [ ]:
# Plot the result from the above tables

plt.rcParams['figure.figsize'] = 20, 10
fig, axs = plt.subplots(nrows=2)
ax0 = reading_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[0])
ax0.set_ylabel('Percentage')
ax0.set_xlabel('')
ax0.set_title('Reading Test Achievement Level Distribution - All Students')
ax0.get_legend().remove()

ax1 = math_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[1])
ax1.set_ylabel('Percentage')
ax1.set_xlabel('Grade')
ax1.set_title('Math Test Achievement Level Distribution - All Students')
ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45))

plt.savefig("Reading&MathTest.png")

plt.show()




### Seperated by Economic Status: Disadvantaged or Not Disadvantaged

In [ ]:
# MAKE NEW DATAFRAMES BASED ON ECONOMIC STATUS WITH ADVANTAGED/DISADVANTAGED STUDENTS
# Student group which has economic disadvantage (Subgroup ID = 31)
df_dis = newdf[newdf['Subgroup ID'] == 31]
df_dis_reading = df_dis[df_dis['Test Id'] == 1] #reading test
df_dis_math = df_dis[df_dis['Test Id'] == 2] #math test

# Student group which doesn't have economic disadvantage (Subgroup ID = 111)
df_adv = newdf[newdf['Subgroup ID'] == 111]
df_adv_reading = df_adv[df_adv['Test Id'] == 1] #reading test
df_adv_math = df_adv[df_adv['Test Id'] == 2] #math test

In [ ]:
# Similarily, get the mean of percentage performance on reading test for each grade 
# Disadvantaged group
dis_reading_mean = df_dis_reading[df_dis_reading.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()
dis_reading_mean.loc['All'] = reading_mean.mean()

# Advantaged group
adv_reading_mean = df_adv_reading[df_adv_reading.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()
adv_reading_mean.loc['All'] = reading_mean.mean()

In [ ]:
plt.rcParams['figure.figsize'] = 20, 10
fig, axs = plt.subplots(nrows=2)
ax0 = dis_reading_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[0])
ax0.set_ylabel('Percentage')
ax0.set_xlabel('')
ax0.set_title('Reading Test Achievement Level Distribution - Economically Disadvantaged Students')
ax0.get_legend().remove()

ax1 = adv_reading_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[1])
ax1.set_ylabel('Percentage')
ax1.set_xlabel('Grade')
ax1.set_title('Math Test Achievement Level Distribution - Not Economically Disadvantaged Students')
ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45))

plt.savefig("Reading&MathAchievement.png")

plt.show()


In [ ]:
# Do the same thing to find MEAN for math test for ADVANTAGED/DISADVANTAGED STUDENTS excluding grade 13 because grade e13 shows all students
dis_math_mean = df_dis_math[df_dis_math.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()
dis_math_mean.loc['All'] = math_mean.mean()

adv_math_mean = df_adv_math[df_adv_math.Grade != 13].groupby('Grade')[['Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met']].mean()
adv_math_mean.loc['All'] = math_mean.mean()

In [ ]:
#plot figures for math test results for economically advantaged and disadvantaged students..
fig, axs = plt.subplots(nrows=2)
ax0 = dis_math_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[0])
ax0.set_ylabel('Percentage')
ax0.set_xlabel('')
ax0.set_title('Math Test Achievement Level Distribution - Economically Disadvantaged Students')
ax0.get_legend().remove()

ax1 = adv_math_mean.plot(kind='bar', stacked=True, rot=0, ax=axs[1])
ax1.set_ylabel('Percentage')
ax1.set_xlabel('Grade')
ax1.set_title('Math Test Achievement Level Distribution - Economically Advantaged Students')
# 4 bbox to anchor 4 element tuple argument
ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45))

plt.savefig("PercentageforMathTest.png")

plt.show()


In [ ]:
pd.__version__